In [21]:
import pandas as pd
df= pd.read_csv('spotify_history.csv')
df['track_url'] = 'https://open.spotify.com/track/' + df['spotify_track_uri']

In [23]:
df.spotify_track_uri.unique()

array(['2J3n32GeLmMjwuAzyhcSNe', '1oHxIPqJyvAYHy0PVrDU98',
       '487OPlneJNni3NWC8SYqhW', ..., '0mWXrr6HQZUhCQG4JMkT2e',
       '2dhwJNH50XJWS3tCKVWKtD', '0mD2pnYVF8DB3oqtY5KM8h'], dtype=object)

In [25]:
# Load CSV file containing track IDs
df = pd.DataFrame(df.spotify_track_uri.unique(), columns=["spotify_track_uri"])

In [27]:
import requests
import pandas as pd

# Spotify API credentials (Replace with your own)
CLIENT_ID = 'f7d77d6ae3f14dc8baebb37b9e758cb1'
CLIENT_SECRET = '29b46d3070974bf1a9837674c9a7d988'

# Function to get Spotify Access Token
def get_spotify_token():
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_response = requests.post(auth_url, {
        'grant_type': 'client_credentials',
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
    })
    auth_data = auth_response.json()
    return auth_data['access_token']

# Function to fetch album cover URL using track ID
def get_album_cover(track_id, token):
    url = f"https://api.spotify.com/v1/tracks/{track_id}"
    response = requests.get(url, headers={'Authorization': f'Bearer {token}'})
    if response.status_code == 200:
        json_data = response.json()
        return json_data['album']['images'][0]['url']
    return None


# Get access token
access_token = get_spotify_token()

In [29]:
# Fetch album covers for each track ID
df["cover_url"] = df["spotify_track_uri"].apply(lambda x: get_album_cover(x, access_token))

IndexError: list index out of range

In [31]:
df

,spotify_track_uri
0,2J3n32GeLmMjwuAzyhcSNe
1,1oHxIPqJyvAYHy0PVrDU98
2,487OPlneJNni3NWC8SYqhW
3,5IyblF777jLZj1vGHG2UD3
4,0GgAAB0ZMllFhbNc3mAodO
...,...
16522,0q1FeGOHG0q6OtAhyNRnEb
16523,15oN81AHUXutUjqvlnjeHE
16524,0mWXrr6HQZUhCQG4JMkT2e
16525,2dhwJNH50XJWS3tCKVWKtD


In [ ]:
# Save updated file
df.to_csv("updated_spotify_data.csv", index=False)

In [19]:
df.columns

Index(['spotify_track_uri'], dtype='object')

In [ ]:
import aiohttp
import asyncio
import pandas as pd
import base64
import requests
import time

# Spotify API credentials
CLIENT_ID = 'f7d77d6ae3f14dc8baebb37b9e758cb1'
CLIENT_SECRET = '29b46d3070974bf1a9837674c9a7d988'

# Function to get a fresh Spotify access token
def get_access_token():
    auth_string = f'{CLIENT_ID}:{CLIENT_SECRET}'
    encoded_auth_string = base64.b64encode(auth_string.encode('ascii')).decode('ascii')

    url = 'https://accounts.spotify.com/api/token'
    headers = {'Authorization': f'Basic {encoded_auth_string}', 'Content-Type': 'application/x-www-form-urlencoded'}
    data = {'grant_type': 'client_credentials'}

    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json().get('access_token')
    else:
        print(f"Failed to get token: {response.status_code}, {response.text}")
        return None

# Fetch token before starting async tasks
TOKEN = get_access_token()

# Function to fetch track image asynchronously
async def fetch_image_url(session, track_id):
    url = f'https://api.spotify.com/v1/tracks/{track_id}'
    headers = {'Authorization': f'Bearer {TOKEN}'}

    async with session.get(url, headers=headers) as response:
        if response.status == 200:
            track_data = await response.json()
            return track_id, track_data['album']['images'][0]['url']
        elif response.status == 429:  # Too Many Requests
            retry_after = int(response.headers.get("Retry-After", 5))
            print(f"Rate limit hit! Waiting {retry_after} seconds...")
            await asyncio.sleep(retry_after)
            return await fetch_image_url(session, track_id)  # Retry
        else:
            print(f"Failed to fetch {track_id}: {response.status}")
            return track_id, None

# Async function to run multiple requests
async def fetch_all_images(track_ids):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_image_url(session, track_id) for track_id in track_ids]
        return await asyncio.gather(*tasks)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Load dataset
#df = pd.read_csv("spotify_tracks.csv")  # Change file name accordingly
unique_track_ids = tracks_id['spotify_track_uri']

# Run async function to fetch images
# Run async function to fetch images
image_results = asyncio.run(fetch_all_images(unique_track_ids))

# Convert to DataFrame
image_df = pd.DataFrame(image_results, columns=['spotify_track_uri', 'image_url'])
tracks_id = tracks_id.merge(image_df, on='spotify_track_uri', how='left')

In [ ]:
# Save the updated dataset
tracks_id.to_csv("spotify_tracks_with_images.csv", index=False)

print("✅ Fetching complete. File saved as 'spotify_tracks_with_images.csv'")

In [ ]:
import requests
import time
import pandas as pd
import base64

# Spotify API credentials
CLIENT_ID = 'f7d77d6ae3f14dc8baebb37b9e758cb1'
CLIENT_SECRET = '29b46d3070974bf1a9837674c9a7d988'


# Function to get a new access token
def get_access_token():
    auth_string = f'{CLIENT_ID}:{CLIENT_SECRET}'
    encoded_auth_string = base64.b64encode(auth_string.encode('ascii')).decode('ascii')

    url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Authorization': f'Basic {encoded_auth_string}',
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    data = {'grant_type': 'client_credentials'}

    response = requests.post(url, headers=headers, data=data)
    
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        print(f"Failed to get token: {response.status_code}, {response.text}")
        return None

# Function to fetch track image
def fetch_image_url(track_id, token):
    url = f'https://api.spotify.com/v1/tracks/{track_id}'
    headers = {'Authorization': f'Bearer {token}'}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        track_data = response.json()
        return track_data['album']['images'][0]['url']
    elif response.status_code == 401:  # Token expired
        print("🔄 Token expired, refreshing...")
        return "TOKEN_EXPIRED"
    elif response.status_code == 429:  # Rate limit hit
        print("🚨 Rate limit hit! Waiting...")
        time.sleep(5)  # Wait 5 seconds and retry
        return fetch_image_url(track_id, token)
    else:
        print(f"⚠️ Failed to fetch {track_id}: {response.status_code}")
        return None

# Load CSV file (Replace with your file path)
df = pd.read_csv('spotify_history.csv')

# Extract unique track IDs
unique_track_ids = df['spotify_track_uri'].unique()

# Get initial access token
token = get_access_token()

# Fetch image URLs
image_data = []
for i, track_id in enumerate(unique_track_ids):
    print(f"Fetching {i+1}/{len(unique_track_ids)}: {track_id}")
    
    image_url = fetch_image_url(track_id, token)
    
    # If token expired, refresh and retry
    if image_url == "TOKEN_EXPIRED":
        token = get_access_token()
        image_url = fetch_image_url(track_id, token)
    
    image_data.append((track_id, image_url))
    
    # Respect API rate limits (Spotify allows ~50-100 requests/sec)
    time.sleep(0.1)  # Wait 100ms between requests

# Convert to DataFrame and Save
image_df = pd.DataFrame(image_data, columns=['spotify_track_uri', 'image_url'])
image_df.to_csv("spotify_images.csv", index=False)

print("✅ Image fetching complete! Saved to 'spotify_images.csv'")

In [ ]:
import requests
import base64

In [ ]:
# Step 1: Get Access Token (You can use your existing access token logic)
CLIENT_ID = 'f7d77d6ae3f14dc8baebb37b9e758cb1'
CLIENT_SECRET = '29b46d3070974bf1a9837674c9a7d988'

auth_string = f'{CLIENT_ID}:{CLIENT_SECRET}'
encoded_auth_string = base64.b64encode(auth_string.encode('ascii')).decode('ascii')

url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {encoded_auth_string}',
    'Content-Type': 'application/x-www-form-urlencoded'
}

data = {'grant_type': 'client_credentials'}
response = requests.post(url, headers=headers, data=data)

if response.status_code == 200:
    token = response.json().get('access_token')
    print(f"Access Token: {token}")
else:
    print(f"Failed to get token: {response.status_code}, {response.text}")
    token = None

In [ ]:
# Function to get the image URL for a track ID
def get_image_url(track_id, token):
    track_url = f'https://api.spotify.com/v1/tracks/{track_id}'
    headers = {
        'Authorization': f'Bearer {token}'
    }
    
    # Make the GET request to fetch track data
    track_response = requests.get(track_url, headers=headers)
    
    if track_response.status_code == 200:
        track_data = track_response.json()
        image_url = track_data['album']['images'][0]['url']
        return image_url
    else:
        return None  # Return None if fetching the image fails

In [ ]:
# Assuming your function get_image_url is defined and df_unique_tracks is the DataFrame
def get_image_url(track_id, token):
    track_url = f'https://api.spotify.com/v1/tracks/{track_id}'
    headers = {
        'Authorization': f'Bearer {token}'
    }
    
    # Make the GET request to fetch track data
    track_response = requests.get(track_url, headers=headers)
    
    if track_response.status_code == 200:
        track_data = track_response.json()
        image_url = track_data['album']['images'][0]['url']
        return image_url
    else:
        return None  # Return None if fetching the image fails

# Apply the get_image_url function to each row in the DataFrame and create a new column 'image_url'
tracks_id['image_url'] = tracks_id['spotify_track_uri'].apply(lambda track_id: get_image_url(track_id, token))

# Display the DataFrame with the new 'image_url' column
print(tracks_id)

In [ ]:
tracks_id

In [ ]:
tracks_id.isnull().sum()

In [ ]:
track_id = '0mD2pnYVF8DB3oqtY5KM8h'  # Replace with the actual track ID
track_url = f'https://api.spotify.com/v1/tracks/{track_id}'

headers = {
    'Authorization': f'Bearer {token}'  # Make sure your token is valid
}

track_response = requests.get(track_url, headers=headers)

if track_response.status_code == 200:
    track_data = track_response.json()
    images = track_data['album']['images']
    if images:  # Check if images array is not empty
        image_url = images[0]['url']
    else:
        image_url = None  # No images available
    print(f"Track Image URL: {image_url}")
else:
    print(f"Failed to fetch track data: {track_response.status_code}")